In [1]:
import os

In [2]:
%pwd

'/Users/jubril/Downloads/AI_PORTFOLIO/ML_MLFlow/research'

In [3]:
os.chdir("../")

In [4]:
%pwd

'/Users/jubril/Downloads/AI_PORTFOLIO/ML_MLFlow'

In [5]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path
    source_URL: str
    local_data_file: Path
    unzip_dir: Path

In [9]:
%pip install -r requirements.txt

Defaulting to user installation because normal site-packages is not writeable
Obtaining file:///Users/jubril/Downloads/AI_PORTFOLIO/ML_MLFlow (from -r requirements.txt (line 15))
  Preparing metadata (setup.py) ... done
  Using cached pandas-2.0.3-cp38-cp38-macosx_10_9_x86_64.whl.metadata (18 kB)
  Using cached mlflow-2.2.2-py3-none-any.whl.metadata (11 kB)
  Using cached notebook-7.3.2-py3-none-any.whl.metadata (10 kB)
  Using cached numpy-1.24.4-cp38-cp38-macosx_10_9_x86_64.whl.metadata (5.6 kB)
  Using cached scikit_learn-1.3.2-cp38-cp38-macosx_10_9_x86_64.whl.metadata (11 kB)
  Using cached matplotlib-3.7.5-cp38-cp38-macosx_10_12_x86_64.whl.metadata (5.7 kB)
  Using cached python_box-6.0.2-cp38-cp38-macosx_10_14_x86_64.whl.metadata (7.7 kB)
  Using cached PyYAML-6.0.2-cp38-cp38-macosx_10_9_x86_64.whl.metadata (2.1 kB)
  Using cached tqdm-4.67.1-py3-none-any.whl.metadata (57 kB)
  Using cached ensure-1.0.2-py2.py3-none-any.whl.metadata (10 kB)
  Using cached joblib-1.4.2-py3-none-an

In [11]:
from src.mlProject.constants import *
from src.mlProject.utils.common import read_yaml, create_directories

In [12]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion

        create_directories([config.root_dir])

        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir 
        )

        return data_ingestion_config

In [13]:
import os
import urllib.request as request
import zipfile
from src.mlProject import logger
from src.mlProject.utils.common import get_size

In [14]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config


    
    def download_file(self):
        if not os.path.exists(self.config.local_data_file):
            filename, headers = request.urlretrieve(
                url = self.config.source_URL,
                filename = self.config.local_data_file
            )
            logger.info(f"{filename} download! with following info: \n{headers}")
        else:
            logger.info(f"File already exists of size: {get_size(Path(self.config.local_data_file))}")



    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)
  

In [17]:
try:
    config = ConfigurationManager()
    data_ingestion_config = config.get_data_ingestion_config()
    data_ingestion = DataIngestion(config=data_ingestion_config)
    data_ingestion.download_file()
    data_ingestion.extract_zip_file()
except Exception as e:
    raise e

[2025-02-27 15:10:36,002: INFO: common: yaml file: config/config.yaml loaded successfully]
[2025-02-27 15:10:36,093: INFO: common: yaml file: params.yaml loaded successfully]
[2025-02-27 15:10:36,112: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-02-27 15:10:36,118: INFO: common: created directory at: artifacts]
[2025-02-27 15:10:36,123: INFO: common: created directory at: artifacts/data_ingestion]
[2025-02-27 15:10:37,417: INFO: 2366278714: artifacts/data_ingestion/data.zip download! with following info: 
Connection: close
Content-Length: 23329
Cache-Control: max-age=300
Content-Security-Policy: default-src 'none'; style-src 'unsafe-inline'; sandbox
Content-Type: application/zip
ETag: "c69888a4ae59bc5a893392785a938ccd4937981c06ba8a9d6a21aa52b4ab5b6e"
Strict-Transport-Security: max-age=31536000
X-Content-Type-Options: nosniff
X-Frame-Options: deny
X-XSS-Protection: 1; mode=block
X-GitHub-Request-Id: B114:31FA39:2C9AE1:6692D7:67C0806C
Accept-Ranges: bytes
Date: Thu, 27